In [ ]:
!git clone https://github.com/AlexeyAB/darknet 

In [ ]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!cat Makefile
!make

In [ ]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/drive'):
  drive.mount('/content/drive',force_remount=True)
  
DOWNLOAD_LOCATION = '/content/darknet/data/'
DRIVE_DATASET_FILE = '/content/drive/MyDrive/FBI_dataset.zip'

shutil.copy(DRIVE_DATASET_FILE, DOWNLOAD_LOCATION)


print('Successfully downloaded the dataset')

In [ ]:
!unzip /content/darknet/data/FBI_dataset.zip -d data/ 

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74  

In [ ]:
%cd /content/darknet
!sed -i 's/batch=1/batch=64/g' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=16/g' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/g' cfg/yolov3.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/g' cfg/yolov3.cfg
!sed -i 's/classes=80/classes=3/g' cfg/yolov3.cfg
!sed -i 's/filters=255/filters=24/g' cfg/yolov3.cfg
!sed -i 's/width=416/width=224/g' cfg/yolov3.cfg
!sed -i 's/height=416/height=224/g' cfg/yolov3.cfg



!cat cfg/yolov3.cfg

In [ ]:
import os
import sys

CURRENT_DIR = '/content/darknet/data/img'

def progressBar(value, endvalue, bar_length=20):
  percent = float(value) / endvalue
  arrow = '-' * int(round(percent * bar_length)-1) + '>'
  spaces = ' ' * (bar_length - len(arrow))

  sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
  sys.stdout.flush()

def returnFileList(dirname, extract):
  fileList = []
  filenames = os.listdir(dirname)
  for filename in filenames:
    ext = os.path.splitext(filename)[-1]
    if ext == extract: 
      fileList.append(filename)
  return fileList

fileNames = returnFileList(CURRENT_DIR, ".png")

from sklearn.model_selection import train_test_split
train,test = train_test_split(fileNames, test_size=0.1, random_state=123)
train,valid = train_test_split(train,test_size=0.1,random_state=123)


with open("/content/darknet/data/train.txt","w") as fileList:
  for i, names in enumerate(train):
    fileList.write(CURRENT_DIR+"/"+names+"\n")
    progressBar(i, len(train), bar_length=100)

with open("/content/darknet/data/valid.txt","w") as fileList:
  for i, names in enumerate(valid):
    fileList.write(CURRENT_DIR+"/"+names+"\n")
    progressBar(i, len(valid), bar_length=100)

with open("/content/darknet/data/test.txt","w") as fileList:
  for i, names in enumerate(test):
    fileList.write(CURRENT_DIR+"/"+names+"\n")
    progressBar(i, len(test), bar_length=100)

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3.cfg darknet53.conv.74 -dont_show

In [ ]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
  
YOLO_BACKUP = '/content/darknet/backup/yolov3_last.weights' 
DRIVE_DIR = '/content/drive/MyDrive/weight' 

shutil.copy(YOLO_BACKUP, DRIVE_DIR)

print('Saved training data to drive at: ' + DRIVE_DIR)

In [ ]:
%cd /content/darknet

In [ ]:
!./darknet detector map  data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights

In [ ]:
!./darknet detector test data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights data/fire2.jpg -dont_show

In [ ]:
def display_image(file_path = '/content/darknet/predictions.jpg'):
    import cv2
    import matplotlib.pyplot as plt
    import os.path

    if os.path.exists(file_path):
      img = cv2.imread(file_path)
      show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
      plt.imshow(show_img)
    else:
      print('failed to open file')
    
display_image()

In [ ]:
def upload():
  from google.colab import files
  upload = files.upload()
  for name, data in upload.items():
    with open(name, 'wb') as f:
      f.write(data)
      print('saved file', name)

In [ ]:
upload()

In [ ]:
!./darknet detector test data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights -dont_show -out_filename yolo_video.mp4

In [ ]:
import cv2
import numpy as np
import time # -- 프레임 계산을 위해 사용


vedio_path = '/content/darknet/data/fire.mp4' #-- 사용할 영상 경로
min_confidence = 0.5

def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.8, fy=0.8)
    height, width, channels = img.shape
    cv2.imshow("Original Image", img)

    #-- 창 크기 설정
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    #-- 탐지한 객체의 클래스 예측 
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > min_confidence:
                # 탐지한 객체 박싱
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
            print(i, label)
            color = colors[i] #-- 경계 상자 컬러 설정 / 단일 생상 사용시 (255,255,255)사용(B,G,R)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    end_time = time.time()
    process_time = end_time - start_time
    # print("=== A frame took {:.3f} seconds".format(process_time))
    # cv2.imshow("YOLO test", img)
    
#-- yolo 포맷 및 클래스명 불러오기
model_file = '/ontent/darknet/backup/yolov3_last.weights' #-- 본인 개발 환경에 맞게 변경할 것
config_file = '/content/darknet/cfg/yolov3.cfg' #-- 본인 개발 환경에 맞게 변경할 것
net = cv2.dnn.readNet(model_file, config_file)


#-- 클래스(names파일) 오픈 / 본인 개발 환경에 맞게 변경할 것
classes = []
with open("/content/darknet/data/obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

#-- 비디오 활성화
cap = cv2.VideoCapture(vedio_path) #-- 웹캠 사용시 vedio_path를 0 으로 변경
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)
    #-- q 입력시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()